In [1]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def list_wav_files_in_directory(directory):
    return sorted([os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".wav")])

def save_spectrogram_as_image(filename, output_dir, counter):
    y, sr = librosa.load(filename)
    S = np.abs(librosa.stft(y, hop_length=512))
    S_db = librosa.amplitude_to_db(S, ref=np.max)
    
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_db, sr=sr, hop_length=512, x_axis=None, y_axis=None)
    plt.axis('off')
    plt.tight_layout(pad=0)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    plt.savefig(os.path.join(output_dir, f'{counter}.png'), bbox_inches='tight', pad_inches=0)
    plt.close()

def get_next_counter(output_dir):
    existing_files = [f for f in os.listdir(output_dir) if f.endswith('.png')]
    if existing_files:
        max_number = max(int(f.split('.')[0]) for f in existing_files)
        return max_number + 1
    else:
        return 0

def delete_long_audio_files(directory, max_duration=25): #24 Sekunden hier festgelegt
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        
        if filepath.lower().endswith('.wav'):
            try:
                y, sr = librosa.load(filepath, sr=None)
                duration = librosa.get_duration(y=y, sr=sr)
                
                if duration > max_duration:
                    os.remove(filepath)
                    print(f"Deleted {filepath} - Duration: {duration} seconds")
            except Exception as e:
                print(f"Failed to process {filepath}: {e}")

def process_files_for_split(split, label, input_base_dir, output_base_dir, max_duration=25):
    input_dir = os.path.join(input_base_dir, label, split)
    output_dir = os.path.join(output_base_dir, split, label)
    
    delete_long_audio_files(input_dir, max_duration)
    
    files = list_wav_files_in_directory(input_dir)
    counter = get_next_counter(output_dir)
    
    for file in files:
        save_spectrogram_as_image(file, output_dir, counter)
        counter += 1

input_base_dir = 'ALL_segments'
output_base_dir = 'spectrograms'
splits = ['train', 'val', 'test']
labels = ['alu_nut_io', 'alu_nut_nio', 'alu_ohne_io', 'alu_ohne_nio', 'kunststoff_nut_io', 'kunststoff_nut_nio', 'kunststoff_ohne_io', 'kunststoff_ohne_nio']

for split in splits:
    for label in labels:
        process_files_for_split(split, label, input_base_dir, output_base_dir)

print('Spektrogramme wurden erfolgreich generiert und gespeichert.')


Spektrogramme wurden erfolgreich generiert und gespeichert.
